## Load packages

In [1]:
%%capture
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [3]:
%%capture
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

In [27]:
!pip install igraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.1 MB/s eta 0:00:00


In [28]:
import graph_tool.all as gt
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import networkx as nx #
from igraph import * #

## Load data

In [ ]:
g=Graph.Read_Gml("/content/network.gml")

In [25]:
g=nx.read_gml("/content/network.gml")


NetworkXError: ignored

In [ ]:
g = gt.collection.ns["twitter_higgs/retweet"]

In [ ]:
print(g)

<Graph object, directed, with 256491 vertices and 328132 edges, 2 internal vertex properties, 1 internal edge property, 5 internal graph properties, at 0x7f644763a500>


In [ ]:
g.list_properties()

## In and out degree distribution

In [ ]:
in_hist = gt.vertex_hist(g, "in")

out_hist = gt.vertex_hist(g, "out")

In [ ]:
y = in_hist[0]
err = np.sqrt(in_hist[0])
plt.errorbar(in_hist[1][:-1], in_hist[0], fmt="o", yerr=err,
        label="in")

y = out_hist[0]
err = np.sqrt(out_hist[0])
plt.errorbar(out_hist[1][:-1], out_hist[0], fmt="o", yerr=err,
        label="out")

plt.yscale("log")
plt.xscale("log")

plt.xlabel("$k$")
plt.ylabel("$NP(k_{in})$")

plt.tight_layout()
plt.legend()

## Select a subgraph for visualization

In [ ]:
u = gt.extract_largest_component(g)

print(u)

u = gt.Graph(u, prune=True)

print(u)

In [ ]:
# spring layout
pos = gt.sfdp_layout(u)

In [ ]:
gt.graph_draw(u, pos)

In [ ]:
pos_fr = gt.fruchterman_reingold_layout(u, n_iter=100)

In [ ]:
gt.graph_draw(u, pos_fr)

In [ ]:
gt.graph_draw(u, pos=u.vp["_pos"])

In [ ]:
import geopandas as gpd

countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:
pos_tree = gt.radial_tree_layout(u, 0)

In [ ]:
gt.graph_draw(u, pos_tree)

In [ ]:
maximum = max(u.get_in_degrees(u.get_vertices()))

print(np.where(u.get_in_degrees(u.get_vertices()) == maximum))
print(u.get_in_degrees(u.get_vertices()))

In [ ]:
pos_tree = gt.radial_tree_layout(u, 1)

In [ ]:
gt.graph_draw(u, pos_tree)

In [ ]:
state = gt.minimize_nested_blockmodel_dl(u)

t = gt.get_hierarchy_tree(state)[0]

tpos = pos = gt.radial_tree_layout(t, t.vertex(t.num_vertices() - 1, use_index=False), weighted=True)

cts = gt.get_hierarchy_control_points(u, t, tpos)

pos = u.own_property(tpos)

b = state.levels[0].b

shape = b.copy()

shape.a %= 14

gt.graph_draw(u, pos=pos, vertex_fill_color=b, vertex_shape=shape, edge_control_points=cts,

              edge_color=[0, 0, 0, 0.3], vertex_anchor=0)

## Shortest paths

In [ ]:
dist = gt.shortest_distance(u, source=u.vertex(1))

print(dist.a)

plt.hist(dist.a)

In [ ]:
hist, bin_edges = gt.distance_histogram(u)
print(hist)
plt.bar(bin_edges[:-1], hist, width=1)

In [ ]:
dist = gt.shortest_distance(u)

average_path_length = gt.vertex_average(u, dist)
print(np.mean(average_path_length))

## Percolation

In [ ]:
edges = sorted([(e.source(), e.target()) for e in u.edges()],
               key=lambda e: e[0].in_degree() * e[1].in_degree())

sizes, comp = gt.edge_percolation(u, edges)

np.random.shuffle(edges)

sizes2, comp = gt.edge_percolation(u, edges)

plt.plot(sizes, label="Targeted")

plt.plot(sizes2, label="Random")

plt.legend()

## k-core decomposition

In [ ]:
kcore = gt.kcore_decomposition(u)
gt.graph_draw(u, vertex_fill_color=kcore, vertex_text=kcore)

In [ ]:
gt.pseudo_diameter(u)

## Centrality measures

In [ ]:
pr = gt.pagerank(u)

gt.graph_draw(u, pos=u.vp["_pos"], vertex_fill_color=pr,

              vertex_size=gt.prop_to_size(pr, mi=5, ma=15),

              vorder=pr, vcmap=matplotlib.cm.gist_heat)

In [ ]:
vp, ep = gt.betweenness(u)

gt.graph_draw(u, pos=u.vp["_pos"], vertex_fill_color=vp,

              vertex_size=gt.prop_to_size(vp, mi=5, ma=15),

              edge_pen_width=gt.prop_to_size(ep, mi=0.5, ma=5),

              vcmap=matplotlib.cm.gist_heat,

              vorder=vp)

In [ ]:
c = gt.closeness(u)

gt.graph_draw(u, pos=u.vp["_pos"], vertex_fill_color=c,

              vertex_size=gt.prop_to_size(c, mi=0.5, ma=5),

              vcmap=matplotlib.cm.gist_heat,

              vorder=c)

In [ ]:
ee, x = gt.eigenvector(u, u.ep["weight"])

gt.graph_draw(u, pos=u.vp["_pos"], vertex_fill_color=x,

              vertex_size=gt.prop_to_size(x, mi=5, ma=15),

              vcmap=matplotlib.cm.gist_heat,

              vorder=x)

In [ ]:
gt.graph_draw(u, pos=pos_fr, vertex_fill_color=x,

              vertex_size=gt.prop_to_size(x, mi=5, ma=15),

              vcmap=matplotlib.cm.gist_heat,

              vorder=x)

In [ ]:
clust = gt.local_clustering(u)

print(gt.vertex_average(u, clust))

In [ ]:
print(clust.a)

In [ ]:
out_degs = u.get_out_degrees(u.get_vertices())

In [ ]:
plt.scatter(out_degs, clust.a, alpha=0.5)

In [ ]:
in_degs = u.get_in_degrees(u.get_vertices())

In [ ]:
plt.scatter(in_degs, clust.a, alpha=0.5)

## Blockmodel

In [ ]:
state = gt.minimize_blockmodel_dl(u)

state.draw(pos=u.vp["_pos"], vertex_shape=state.get_blocks())

In [ ]:
state.draw(pos=pos_fr, vertex_shape=state.get_blocks())

In [ ]:
state.draw(pos=pos_fr, vertex_shape=state.get_blocks(), vertex_size=gt.prop_to_size(x, mi=5, ma=25))

In [ ]:
state2 = gt.minimize_nested_blockmodel_dl(u)

state2.draw()

In [ ]:
print(state.get_blocks().a)

In [ ]:
print(u.vp["name"].a)

In [ ]:
u.save("retweets.graphml")

In [ ]:
g_wiki = gt.collection.ns["wikipedia_link/de"]

In [ ]:
print(g_wiki)

In [ ]:
g_wiki.list_properties()

In [ ]:
g_gov = gt.collection.ns["us_agencies/alabama"]

In [ ]:
g_gov.list_properties()

In [ ]:
g_email = gt.collection.ns["email_enron"]

In [ ]:
g_email.list_properties()

In [ ]:
g_faculty = gt.collection.ns["faculty_hiring_us/academia"]

In [ ]:
g_faculty.list_properties()

In [ ]:
g_cop = gt.collection.ns["copenhagen/sms"]

In [ ]:
g_cop.list_properties()